# Multi-Lingual Vanila RAG with Together.ai and Chroma DB: No LangChain or LamaIndex

## Tested Languages: EN, URDU

## Table of Contents
1. Document Processing and Indexing
2. Setting Up ChromaDB
3. Inserting Data into ChromaDB
4. Semantic Search on ChromaDB
5. Combining ChromaDB and together.ai for RAG
6. Creating Conversational RAG with Memory


### What is Retrieval Augmented Generation (RAG)?

RAG is a technique that enhances language models by combining them with a retrieval system. It allows the model to access and utilize external knowledge when generating responses.

## Installing Necessary Libraries

In [1]:
%pip install -qU chromadb openai pypdf2 python-docx python-multipart sentence-transformers==3.1.0 langchain-huggingface together

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Document Processing and Indexing

### Functions to read file contents

In [2]:
import docx
import PyPDF2
import os
def read_text_file(file_path: str):
    """Read content from a txt file"""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def read_pdf_file(file_path: str):
    """Read content from a PDF file"""
    text = ""
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
    return text

def read_docx_file(file_path: str):
    """Read content from a docx file"""
    doc = docx.Document(file_path)
    return "\n".join([paragraph.text for paragraph in doc.paragraphs])

In [3]:
def read_document(file_path: str):
    """Read document content based on file extension"""
    _, file_extension = os.path.splitext(file_path)
    file_extension = file_extension.lower()

    if file_extension == '.txt':
        return read_text_file(file_path)
    elif file_extension == '.pdf':
        return read_pdf_file(file_path)
    elif file_extension == '.docx':
        return read_docx_file(file_path)
    else:
        raise ValueError(f"Unsupported file format: {file_extension}")

#NextView_Technologies_Profile_Urdu.docx
#TechNova_Company_Profile.txt
text = read_document("docs/NextView_Technologies_Profile_Urdu.docx")
print(text)


کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز
قیام: 2010
مرکز: لاہور، پاکستان

ہمارے بارے میں:
نیکسٹ ویو ٹیکنالوجیز پاکستان کی ایک معروف ٹیکنالوجی کمپنی ہے جو جدید سافٹ ویئر اور ڈیجیٹل حل فراہم کرتی ہے۔ 
ہماری ٹیم کا مقصد کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرنا ہے۔

اہم خدمات:
1. ویب اور موبائل ایپلیکیشن ڈویلپمنٹ
2. ڈیجیٹل مارکیٹنگ اور برانڈنگ
3. ای کامرس سلوشنز
4. کلاؤڈ بیسڈ ایپلیکیشنز
5. آرٹیفیشل انٹیلیجنس پر مبنی پراڈکٹس

اہم کلائنٹس:
1. معروف ریٹیل اسٹورز نیٹ ورک
2. تعلیمی ادارے اور یونیورسٹیز
3. فیشن اور ٹیکسٹائل انڈسٹری
4. صحت کے شعبے میں خدمات فراہم کرنے والی تنظیمیں

ایوارڈز اور اعزازات:
- بہترین ڈیجیٹل سروسز کمپنی 2021
- آئی ٹی انوویشن ایوارڈ 2020
- پاکستان کی ٹاپ 100 ٹیکنالوجی کمپنیز میں شامل

رابطے کی معلومات:
ویب سائٹ: www.nextviewtech.com
ای میل: info@nextviewtech.com
فون: +92 (42) 34567890
پتہ: 456 ڈیجیٹل ٹاور، مین بلیوارڈ، لاہور، پاکستان



### Chunking

In [4]:
def split_text(text: str, chunk_size: int = 500):
    """Split text into chunks"""
    sentences = text.replace('\n', ' ').split('. ')
    chunks = []
    current_chunk = []
    current_size = 0

    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue

        if not sentence.endswith('.'):
            sentence += '.'

        sentence_size = len(sentence)

        if current_size + sentence_size > chunk_size and current_chunk:
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_size = sentence_size
        else:
            current_chunk.append(sentence)
            current_size += sentence_size

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

chunks = split_text(text)
print(chunks[1])

معروف ریٹیل اسٹورز نیٹ ورک 2. تعلیمی ادارے اور یونیورسٹیز 3. فیشن اور ٹیکسٹائل انڈسٹری 4. صحت کے شعبے میں خدمات فراہم کرنے والی تنظیمیں  ایوارڈز اور اعزازات: - بہترین ڈیجیٹل سروسز کمپنی 2021 - آئی ٹی انوویشن ایوارڈ 2020 - پاکستان کی ٹاپ 100 ٹیکنالوجی کمپنیز میں شامل  رابطے کی معلومات: ویب سائٹ: www.nextviewtech.com ای میل: info@nextviewtech.com فون: +92 (42) 34567890 پتہ: 456 ڈیجیٹل ٹاور، مین بلیوارڈ، لاہور، پاکستان.


In [5]:
len(chunks)

2

## Setting Up ChromaDB

In [6]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

In [ ]:
# Initialize ChromaDB client
client = chromadb.PersistentClient(path="./chroma_db")

# Use Jina mutlilingual embeddings for embedding our data
jinaai_ef = embedding_functions.JinaEmbeddingFunction(
                api_key="Jina API key",
                model_name="jina-embeddings-v3"
            )

collection = client.get_or_create_collection(name="documents_collection", embedding_function=jinaai_ef)

/Users/abubakarsiddique/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Inserting Data into ChromaDB

In [8]:
def process_document(file_path: str):
    """Process a single document and prepare it for ChromaDB"""
    try:
        # Read the document
        content = read_document(file_path)

        # Split into chunks
        chunks = split_text(content)

        # Prepare metadata
        file_name = os.path.basename(file_path)
        metadatas = [{"source": file_name, "chunk": i} for i in range(len(chunks))]
        ids = [f"{file_name}_chunk_{i}" for i in range(len(chunks))]

        return ids, chunks, metadatas
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return [], [], []

def add_to_collection(collection, ids, texts, metadatas):
    """Add documents to collection in batches"""
    if not texts:
        return

    batch_size = 100
    for i in range(0, len(texts), batch_size):
        end_idx = min(i + batch_size, len(texts))
        collection.add(
            documents=texts[i:end_idx],
            metadatas=metadatas[i:end_idx],
            ids=ids[i:end_idx]
        )

def process_and_add_documents(collection, folder_path: str):
      files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

      for file_path in files:
        print(f"Processing {os.path.basename(file_path)}...")
        ids, texts, metadatas = process_document(file_path)
        add_to_collection(collection, ids, texts, metadatas)
        print(f"Added {len(texts)} chunks to collection")

In [9]:
process_and_add_documents(collection, "./docs")

Processing .DS_Store...
Error processing ./docs/.DS_Store: Unsupported file format: 
Added 0 chunks to collection
Processing NextView_Technologies_Profile_Urdu.docx...
Added 2 chunks to collection
Processing TechNova_Company_Profile.txt...
Added 3 chunks to collection


## Semantic Search on ChromaDB

In [10]:
def semantic_search(collection, query: str, n_results: int = 2):
    """Perform semantic search on the collection"""
    return collection.query(
        query_texts=[query],
        n_results=n_results
    )

In [11]:
#query = "When was Technova founded?"
query = "نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟"
results = semantic_search(collection, query)
results

{'ids': [['NextView_Technologies_Profile_Urdu.docx_chunk_0',
   'NextView_Technologies_Profile_Urdu.docx_chunk_1']],
 'embeddings': None,
 'documents': [['کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز قیام: 2010 مرکز: لاہور، پاکستان  ہمارے بارے میں: نیکسٹ ویو ٹیکنالوجیز پاکستان کی ایک معروف ٹیکنالوجی کمپنی ہے جو جدید سافٹ ویئر اور ڈیجیٹل حل فراہم کرتی ہے۔  ہماری ٹیم کا مقصد کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرنا ہے۔  اہم خدمات: 1. ویب اور موبائل ایپلیکیشن ڈویلپمنٹ 2. ڈیجیٹل مارکیٹنگ اور برانڈنگ 3. ای کامرس سلوشنز 4. کلاؤڈ بیسڈ ایپلیکیشنز 5. آرٹیفیشل انٹیلیجنس پر مبنی پراڈکٹس  اہم کلائنٹس: 1.',
   'معروف ریٹیل اسٹورز نیٹ ورک 2. تعلیمی ادارے اور یونیورسٹیز 3. فیشن اور ٹیکسٹائل انڈسٹری 4. صحت کے شعبے میں خدمات فراہم کرنے والی تنظیمیں  ایوارڈز اور اعزازات: - بہترین ڈیجیٹل سروسز کمپنی 2021 - آئی ٹی انوویشن ایوارڈ 2020 - پاکستان کی ٹاپ 100 ٹیکنالوجی کمپنیز میں شامل  رابطے کی معلومات: ویب سائٹ: www.nextviewtech.com ای میل: info@nextviewtech.com فو

In [12]:
def print_search_results(results):
    """Print formatted search results"""
    print("\nSearch Results:\n" + "-" * 50)

    for i in range(len(results['documents'][0])):
        doc = results['documents'][0][i]
        meta = results['metadatas'][0][i]
        print(f"\nResult {i + 1}: Source: {meta['source']}, Chunk {meta['chunk']}")
        print(f"Content: {doc}\n")

print_search_results(results)


Search Results:
--------------------------------------------------

Result 1: Source: NextView_Technologies_Profile_Urdu.docx, Chunk 0
Content: کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز قیام: 2010 مرکز: لاہور، پاکستان  ہمارے بارے میں: نیکسٹ ویو ٹیکنالوجیز پاکستان کی ایک معروف ٹیکنالوجی کمپنی ہے جو جدید سافٹ ویئر اور ڈیجیٹل حل فراہم کرتی ہے۔  ہماری ٹیم کا مقصد کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرنا ہے۔  اہم خدمات: 1. ویب اور موبائل ایپلیکیشن ڈویلپمنٹ 2. ڈیجیٹل مارکیٹنگ اور برانڈنگ 3. ای کامرس سلوشنز 4. کلاؤڈ بیسڈ ایپلیکیشنز 5. آرٹیفیشل انٹیلیجنس پر مبنی پراڈکٹس  اہم کلائنٹس: 1.


Result 2: Source: NextView_Technologies_Profile_Urdu.docx, Chunk 1
Content: معروف ریٹیل اسٹورز نیٹ ورک 2. تعلیمی ادارے اور یونیورسٹیز 3. فیشن اور ٹیکسٹائل انڈسٹری 4. صحت کے شعبے میں خدمات فراہم کرنے والی تنظیمیں  ایوارڈز اور اعزازات: - بہترین ڈیجیٹل سروسز کمپنی 2021 - آئی ٹی انوویشن ایوارڈ 2020 - پاکستان کی ٹاپ 100 ٹیکنالوجی کمپنیز میں شامل  رابطے کی معلومات: و

In [13]:
def get_context_with_sources(results):
    """Get a combined context and formatted sources from search results."""
    # Combine the document chunks into a single context
    context = "\n\n".join(results['documents'][0])

    # Format the sources with metadata information
    sources = [f"{meta['source']} (chunk {meta['chunk']})" for meta in results['metadatas'][0]]

    return context, sources

context, sources = get_context_with_sources(results)
print(context)

کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز قیام: 2010 مرکز: لاہور، پاکستان  ہمارے بارے میں: نیکسٹ ویو ٹیکنالوجیز پاکستان کی ایک معروف ٹیکنالوجی کمپنی ہے جو جدید سافٹ ویئر اور ڈیجیٹل حل فراہم کرتی ہے۔  ہماری ٹیم کا مقصد کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرنا ہے۔  اہم خدمات: 1. ویب اور موبائل ایپلیکیشن ڈویلپمنٹ 2. ڈیجیٹل مارکیٹنگ اور برانڈنگ 3. ای کامرس سلوشنز 4. کلاؤڈ بیسڈ ایپلیکیشنز 5. آرٹیفیشل انٹیلیجنس پر مبنی پراڈکٹس  اہم کلائنٹس: 1.

معروف ریٹیل اسٹورز نیٹ ورک 2. تعلیمی ادارے اور یونیورسٹیز 3. فیشن اور ٹیکسٹائل انڈسٹری 4. صحت کے شعبے میں خدمات فراہم کرنے والی تنظیمیں  ایوارڈز اور اعزازات: - بہترین ڈیجیٹل سروسز کمپنی 2021 - آئی ٹی انوویشن ایوارڈ 2020 - پاکستان کی ٹاپ 100 ٹیکنالوجی کمپنیز میں شامل  رابطے کی معلومات: ویب سائٹ: www.nextviewtech.com ای میل: info@nextviewtech.com فون: +92 (42) 34567890 پتہ: 456 ڈیجیٹل ٹاور، مین بلیوارڈ، لاہور، پاکستان.


## Combining ChromaDB and Together.ai for RAG

## Together.ai Chat API

In [ ]:
import os
os.environ["TOGETHER_API_KEY"] = ""
from together import Together
client = Together()

In [15]:
completion = client.chat.completions.create(
    model="meta-llama/Llama-Vision-Free",
    messages=[
        {
                "role": "user",
                "content": [
                        {
                                "type": "text",
                                "text": "تم کیا کر رہے ہو"
                        }
                ]
        }
],
    max_tokens=512,
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["<|eot_id|>","<|eom_id|>"],
    update_at="2024-11-21T13:29:41.386Z",
    stream=True
)


In [16]:
for chunk in completion:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta
        if delta.content:
            print(delta.content, end="", flush=True)


میں اس سوال کا جواب دینے کی کوشش کر رہا ہوں

Functions to get prompt and Together.ai Response

In [17]:
def get_prompt(query: str, context: str):
    """Prompt for Response Generation"""
    prompt = f"""Based on the following context, please answer the question.
    If the answer cannot be derived from the context, say "I cannot answer this based on the provided context."

    Context:
    {context}

    Question: {query}

    Answer:"""

    return prompt

def generate_response(query: str, context: str):
    """Generate a response using OpenAI"""

    prompt = get_prompt(query, context)
    response = client.chat.completions.create(
        model="meta-llama/Llama-Vision-Free",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=512,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stream=True
    )

    return response

## Perform RAG query

In [18]:
def rag_query(collection, query: str, n_chunks: int = 2):
    """Perform RAG query: retrieve relevant chunks and generate answer"""
    # Get relevant chunks
    results = semantic_search(collection, query, n_chunks)
    context, sources = get_context_with_sources(results)

    # Generate response
    response = generate_response(query, context)

    return response, sources

In [19]:
query = "When was Technova founded?"
query = "نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟"
response, sources = rag_query(collection, query)

# Print results
print("\nQuery:", query)
print("\nAnswer:")
for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta
        if delta.content:
            print(delta.content, end="", flush=True)
print("\nSources used:")
for source in sources:
    print(f"- {source}")


Query: نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟

Answer:
2010 میں نیکسٹ ویو ٹیکنالوجیز کی بنیاد رکھی گئی۔
Sources used:
- NextView_Technologies_Profile_Urdu.docx (chunk 0)
- NextView_Technologies_Profile_Urdu.docx (chunk 1)


## Creating Conversational RAG with Memory

In [20]:
import uuid
from datetime import datetime
import json

In [21]:
# In-memory conversation store
conversations = {}

def create_session():
    """Create a new conversation session"""
    session_id = str(uuid.uuid4())
    conversations[session_id] = []
    return session_id

def add_message(session_id: str, role: str, content: str):
    """Add a message to the conversation history"""
    if session_id not in conversations:
        conversations[session_id] = []

    conversations[session_id].append({
        "role": role,
        "content": content,
        "timestamp": datetime.now().isoformat()
    })

In [22]:
def get_conversation_history(session_id: str, max_messages: int = None):
    """Get conversation history for a session"""
    if session_id not in conversations:
        return []

    history = conversations[session_id]
    if max_messages:
        history = history[-max_messages:]

    return history

def format_history_for_prompt(session_id: str, max_messages: int = 5):
    """Format conversation history for inclusion in the prompt"""
    history = get_conversation_history(session_id, max_messages)
    formatted_history = ""

    for msg in history:
        role = "Human" if msg["role"] == "user" else "Assistant"
        formatted_history += f"{role}: {msg['content']}\n\n"

    return formatted_history

## Contextualizing the question

First Question : نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟

Follow up : اس کی اہم خدمات کیا ہیں؟

contextualize_query : Next View Technologies کی اہم خدمات کیا ہیں؟

In [23]:
def contextualize_query(query, conversation_history, client):
    """Generate a standalone query based on conversation history and the latest user query."""
    # Define the system prompt for context-aware question reformulation
    contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )

    # Make the API call to rephrase the question into a standalone format
    completion = client.chat.completions.create(
        model="meta-llama/Llama-Vision-Free",
        messages=[
            {"role": "system", "content": contextualize_q_system_prompt},
            {"role": "user", "content": f"Chat history:\n{conversation_history}\n\nQuestion:\n{query}"}
        ],
        max_tokens=512,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
    )

    # Return the standalone query reformulated by the model
    return completion.choices[0].message.content


In [24]:
def get_prompt(context, conversation_history, query):
  prompt = f"""Based on the following context and conversation history, please provide a relevant and contextual response.
    If the answer cannot be derived from the context, only use the conversation history or say "I cannot answer this based on the provided information."

    Context from documents:
    {context}

    Previous conversation:
    {conversation_history}

    Human: {query}

    Assistant:"""
  return prompt

In [25]:
# Updated generate response function with conversation history also passed for Chatbot Memory
def generate_response(query: str, context: str, conversation_history: str = ""):
    """Generate a response using Together.ai with conversation history"""
    prompt = get_prompt(context, conversation_history, query)
    # print(prompt)

    response = client.chat.completions.create(
        model="meta-llama/Llama-Vision-Free",
        messages=[
            {"role": "system", "content": prompt}
        ],
        max_tokens=512,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
    )

    return response.choices[0].message.content

In [26]:
def conversational_rag_query(
    collection,
    query: str,
    session_id: str,
    n_chunks: int = 2
):
    """Perform RAG query with conversation history"""
    # Get conversation history
    conversation_history = format_history_for_prompt(session_id)

    # Handle follo up questions
    query = contextualize_query(query, conversation_history, client)
    print("Contextualized Query:", query)

    # Get relevant chunks
    context, sources = get_context_with_sources(
        semantic_search(collection, query, n_chunks)
    )
    print("Context:", context)
    print("Sources:", sources)


    response = generate_response(query, context, conversation_history)

    # Add to conversation history
    add_message(session_id, "user", query)
    add_message(session_id, "assistant", response)

    return response, sources

### Conversation 1

In [27]:
session_id = create_session()
print(session_id)

51b6ce6b-ab9b-4f3e-ba9b-175e0e414c85


In [28]:
query = "نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟"
response, sources = conversational_rag_query(
            collection,
            query,
            session_id
)
print(response)

Contextualized Query: NextView Technologies کی بنیاد کب رکھی گئی؟
Context: معروف ریٹیل اسٹورز نیٹ ورک 2. تعلیمی ادارے اور یونیورسٹیز 3. فیشن اور ٹیکسٹائل انڈسٹری 4. صحت کے شعبے میں خدمات فراہم کرنے والی تنظیمیں  ایوارڈز اور اعزازات: - بہترین ڈیجیٹل سروسز کمپنی 2021 - آئی ٹی انوویشن ایوارڈ 2020 - پاکستان کی ٹاپ 100 ٹیکنالوجی کمپنیز میں شامل  رابطے کی معلومات: ویب سائٹ: www.nextviewtech.com ای میل: info@nextviewtech.com فون: +92 (42) 34567890 پتہ: 456 ڈیجیٹل ٹاور، مین بلیوارڈ، لاہور، پاکستان.

کمپنی کا نام: نیکسٹ ویو ٹیکنالوجیز قیام: 2010 مرکز: لاہور، پاکستان  ہمارے بارے میں: نیکسٹ ویو ٹیکنالوجیز پاکستان کی ایک معروف ٹیکنالوجی کمپنی ہے جو جدید سافٹ ویئر اور ڈیجیٹل حل فراہم کرتی ہے۔  ہماری ٹیم کا مقصد کاروباروں کو ڈیجیٹل دور میں ترقی دینے کے لیے تخلیقی اور جدید ٹیکنالوجی پر مبنی خدمات فراہم کرنا ہے۔  اہم خدمات: 1. ویب اور موبائل ایپلیکیشن ڈویلپمنٹ 2. ڈیجیٹل مارکیٹنگ اور برانڈنگ 3. ای کامرس سلوشنز 4. کلاؤڈ بیسڈ ایپلیکیشنز 5. آرٹیفیشل انٹیلیجنس پر مبنی پراڈکٹس  اہم کلائنٹس: 1.
Sources: ['Ne

In [29]:
query = "اس کی اہم خدمات کیا ہیں؟"
response, sources = conversational_rag_query(
            collection,
            query,
            session_id
)
print(response)

Contextualized Query: NextView Technologies کی اہم خدمات کیا ہیں؟
Context: معروف ریٹیل اسٹورز نیٹ ورک 2. تعلیمی ادارے اور یونیورسٹیز 3. فیشن اور ٹیکسٹائل انڈسٹری 4. صحت کے شعبے میں خدمات فراہم کرنے والی تنظیمیں  ایوارڈز اور اعزازات: - بہترین ڈیجیٹل سروسز کمپنی 2021 - آئی ٹی انوویشن ایوارڈ 2020 - پاکستان کی ٹاپ 100 ٹیکنالوجی کمپنیز میں شامل  رابطے کی معلومات: ویب سائٹ: www.nextviewtech.com ای میل: info@nextviewtech.com فون: +92 (42) 34567890 پتہ: 456 ڈیجیٹل ٹاور، مین بلیوارڈ، لاہور، پاکستان.

Custom Software Development 3. Data Analytics & Business Intelligence 4. Cloud Computing Services 5. Cybersecurity Solutions  Key Clients: 1. Global Retail Corporation 2. HealthTech Innovators 3. FinTech Leaders Inc. 4.
Sources: ['NextView_Technologies_Profile_Urdu.docx (chunk 1)', 'TechNova_Company_Profile.txt (chunk 1)']
NextView Technologies کی اہم خدمات میں کастوم سافٹ ویئر ڈویلپمنٹ، ڈیٹا اینالٹکس اور بزنس انٹیلیجنس، کلاؤڈ کمپیوٹنگ سروسز، سائبر سیکیورٹی سولوشنز شامل ہیں۔


In [30]:
conversations

{'51b6ce6b-ab9b-4f3e-ba9b-175e0e414c85': [{'role': 'user',
   'content': 'NextView Technologies کی بنیاد کب رکھی گئی؟',
   'timestamp': '2024-11-22T01:48:28.463992'},
  {'role': 'assistant',
   'content': 'نیکسٹ ویو ٹیکنالوجیز کی بنیاد 2010 میں رکھی گئی تھی۔',
   'timestamp': '2024-11-22T01:48:28.464005'},
  {'role': 'user',
   'content': 'NextView Technologies کی اہم خدمات کیا ہیں؟',
   'timestamp': '2024-11-22T01:48:43.099354'},
  {'role': 'assistant',
   'content': 'NextView Technologies کی اہم خدمات میں کастوم سافٹ ویئر ڈویلپمنٹ، ڈیٹا اینالٹکس اور بزنس انٹیلیجنس، کلاؤڈ کمپیوٹنگ سروسز، سائبر سیکیورٹی سولوشنز شامل ہیں۔',
   'timestamp': '2024-11-22T01:48:43.099369'}]}